# Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as T
import torch.nn.functional as F
from torchvision import models
from torchvision.io import read_image
import pdb


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, utils
from PIL import ImageFile
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.ion()   # interactive mode

In [ ]:
inp=torch.randn([2,3,224,224])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Early Stopping


# Serengeti Dataset

In [ ]:
class SerengetiDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        specie = self.img_labels.iloc[idx, 1]
        descrizione = self.img_labels.iloc[idx, 2:5]
        descrizione = descrizione.to_list()
        descrizione = torch.tensor(descrizione, dtype=torch.float)
        emptyimg = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            specie = self.target_transform(specie)
            descrizione = self.target_transform(descrizione)
            emptyimg = self.target_transform(emptyimg)

        return image, specie, descrizione, emptyimg

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print, loss_based=True):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc_max = 0.0
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        self.loss_based = loss_based

    def __call__(self, val_score, model):
        if self.loss_based:
            score = -val_score
        else:
            score = val_score

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_score, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_score, model)
            self.counter = 0

    def save_checkpoint(self, val_score, model):
        # Saves model when better val_score found
        if self.verbose:
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.4f} --> {val_score:.4f}).  Saving model ...'
                if self.loss_based else f'Validation Accuracy increased ({self.val_acc_max:.4f} --> '
                                        f'{val_score:.4f}).  'f'Saving model ...')
            print()

        torch.save(model.state_dict(), self.path)

        if self.loss_based:
            self.val_loss_min = val_score
        else:
            self.val_acc_max = val_score

    def reset (self):
        self.best_score=None

# Classi per blocks

In [ ]:
class BasicBlock(nn.Module):
  expansion=1

  def __init__(self,inplanes,planes,stride=1,downsample=None):
      super().__init__()
      self.conv1=nn.Conv2d(inplanes,planes,kernel_size=3,stride=stride,padding=1,bias=False)
      self.bn1 = nn.BatchNorm2d(planes)
      self.relu = nn.ReLU(inplace=True)
      self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                     padding=1, bias=False)
      self.bn2 = nn.BatchNorm2d(planes)
      self.downsample = downsample
      self.stride = stride

  def forward(self, x):
      identity = x

      out = self.conv1(x)
      out = self.bn1(out)
      out = self.relu(out)

      out = self.conv2(out)
      out = self.bn2(out)

      if self.downsample is not None:
          identity = self.downsample(x)

      out += identity
      out = self.relu(out)

      return out

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        #self.conv1= nn.Conv2d(in_planes, planes, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


# Classi Branch

In [ ]:
class ClassificationBranch(nn.Module):
  def __init__(self,inplanes,planes):
      super(ClassificationBranch,self).__init__()
      self.conv1=nn.Conv2d(inplanes,planes,kernel_size=3,stride=1,padding=1,bias=False)
      self.bn1=nn.BatchNorm2d(planes)
      self.relu1=nn.ReLU(inplace=True)
      self.pool1=nn.AdaptiveMaxPool2d((1,1))
      self.linear1=nn.Linear(in_features=planes,out_features=64)
      self.bn2 = nn.BatchNorm1d(num_features=64)
      self.relu2=nn.ReLU(inplace=True)
      self.dropout1=nn.Dropout(0.2)
      self.linear2 = nn.Linear(in_features=64, out_features=32)
      self.bn3=nn.BatchNorm1d(num_features=32)
      self.relu3=nn.ReLU(inplace=True)
      self.dropout2=nn.Dropout(0.2)
      self.lineaer3=nn.Linear(in_features=32,out_features=2)
      

  def forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu1(x)
    x=self.pool1(x)
    x=torch.flatten(x,1)
    x=self.linear1(x)
    x=self.bn2(x)
    x=self.relu2(x)
    x=self.dropout1(x)
    x=self.linear2(x)
    x=self.bn3(x)
    x=self.relu3(x)
    x=self.dropout2(x)
    x=self.lineaer3(x)
    
    return x





In [ ]:
class LightClassificationBranch(nn.Module):
  def __init__(self,inplanes,num_classes):
      super(LightClassificationBranch,self).__init__()
      self.conv1=nn.Conv2d(inplanes,512,kernel_size=3,stride=1,padding=1,bias=False)
      self.bn1=nn.BatchNorm2d(512)
      self.relu1=nn.ReLU(inplace=True)
      
      self.conv2=nn.Conv2d(512,2048,kernel_size=3,stride=1,padding=1,bias=False)
      self.bn2=nn.BatchNorm2d(2048)
      self.relu2=nn.ReLU(inplace=True)
      self.pool=nn.AdaptiveMaxPool2d((1,1))
      
      self.linear1=nn.Linear(in_features=2048,out_features=num_classes)
      
      

  def forward(self,x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.relu1(x)
    x=self.conv2(x)
    x=self.bn2(x)
    x=self.relu2(x)
    x=self.pool(x)
    
    x=torch.flatten(x,1)
    out=self.linear1(x)
    
    
    return x,out





# Backbone34

### Backbone34withoutBranches

In [ ]:
class Backbone34(nn.Module):
  def __init__(self,block,block2,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=64,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      #self.LightClassificationBranch1=LightClassificationBranch(64,num_classes=num_classes)
      
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      #self.LightClassificationBranch2=LightClassificationBranch(128,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)
      
      self.inplanes=256
      self.layer31=self._make_layer(block2,128,layers[4],stride=2)
      self.layer41=self._make_layer(block2,256,layers[5],stride=2)
      self.layer51=self._make_layer(block2,512,layers[6],stride=2)
      

     
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      self.fc1=nn.Linear(512*block2.expansion,num_classes)
      
      


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)  
             
      
      x = self.layer2(x) 
            
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1=self.layer51(x1)
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)     # remove 1 X 1 grid and make vector of tensor shape 
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      x2=self.fc2(x2)


      return x1,x2

In [ ]:
backbone34=backbone34()
backbone34

In [ ]:
x1,x2=backbone34(inp)

In [ ]:
backbone34(inp)

(tensor([[ 0.4741, -0.1395, -0.0648, -0.3945,  0.9902, -0.8282, -0.5500,  1.0429,
          -0.6244, -0.0940, -0.1082,  1.0672, -0.3699, -1.0673,  0.3096, -0.1033,
          -0.1673,  0.5847, -0.2893, -2.4647, -0.7236],
         [ 0.3921,  0.1848,  0.5467,  0.2984,  0.6095, -1.1468,  0.2949,  0.5634,
          -0.7786, -0.4431, -1.1236,  1.4900, -0.3843, -0.9994,  0.3966,  0.3101,
          -0.4375,  0.7542, -0.9208, -1.7277, -1.5801]],
        grad_fn=<AddmmBackward0>), tensor([[ 0.1025,  0.3287, -0.5659],
         [ 0.1137,  0.4606, -0.4706]], grad_fn=<AddmmBackward0>))

In [ ]:
x1.shape

torch.Size([2, 21])

In [ ]:
initializeWeights34(backbone34)

In [ ]:
a=backbone34.conv1.weight
b=resnetpre.conv1.weight
torch.equal(a,b)

True

### Backbone34withAddBranches

In [ ]:
backbone34withAddBranche=backbone34withAddBranches()


In [ ]:
out1,out2,x1,x2= backbone34withAddBranche(inp)


In [ ]:
x2.shape

torch.Size([2, 3])

In [ ]:
class ResNetModmodAdd34(nn.Module):
  def __init__(self,block,block2,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=64,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      self.LightClassificationBranch1=LightClassificationBranch(64,num_classes=num_classes)
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      self.LightClassificationBranch2=LightClassificationBranch(128,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)
      
      self.inplanes=256
      self.layer31=self._make_layer(block2,128,layers[4],stride=2)
      self.layer41=self._make_layer(block2,256,layers[5],stride=2)
      self.layer51=self._make_layer(block2,512,layers[6],stride=2)
      

     
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      
      self.fc1=nn.Linear(512*block2.expansion,num_classes)
      
      


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)
      feature_vector1,out1=self.LightClassificationBranch1(x)          # 56x56
      
      
      x = self.layer2(x)  
      feature_vector2,out2=self.LightClassificationBranch2(x)
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1=self.layer51(x1)
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)
      #prova=x1+feature_vector1+feature_vector2
      x1=torch.stack((x1,feature_vector1,feature_vector2),axis=1)
      x1=torch.sum(x1,axis=1)     # remove 1 X 1 grid and make vector of tensor shape 
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      
      x2=self.fc2(x2)


      return out1,out2,x1,x2

### Backbone34withBranches

In [ ]:
class ResNetModmod34(nn.Module):
  def __init__(self,block,block2,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=64,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      self.LightClassificationBranch1=LightClassificationBranch(64,num_classes=num_classes)
      
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      self.LightClassificationBranch2=LightClassificationBranch(128,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)
      
      self.inplanes=256
      self.layer31=self._make_layer(block2,128,layers[4],stride=2)
      self.layer41=self._make_layer(block2,256,layers[5],stride=2)
      self.layer51=self._make_layer(block2,512,layers[6],stride=2)
      

     
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      self.fc1=nn.Linear(512*block2.expansion,num_classes)
      
      


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)  
      feature_vector1,out1=self.LightClassificationBranch1(x)        
      
      x = self.layer2(x) 
      feature_vector2,out2=self.LightClassificationBranch2(x)         
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1=self.layer51(x1)
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)     # remove 1 X 1 grid and make vector of tensor shape 
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      x2=self.fc2(x2)


      return out1,out2,x1,x2

In [ ]:
modelmod34=resnetmodmod34()
initializeWeights34(modelmod34)


In [ ]:
output=modelmod34(inp)
output

In [ ]:
a=modelmod34.conv1.weight
b=resnetpre.conv1.weight
torch.equal(a,b)

True

#### Training

In [ ]:
labels_map = {0: 'baboon',
              1: 'buffalo',
              2: 'cheetah',
              3: 'eland',
              4: 'elephant',
              5: 'empty',
              6: 'gazellegrants',
              7: 'gazellethomsons',
              8: 'giraffe',
              9: 'guineafowl',
              10: 'hartebeest',
              11: 'hyenaspotted',
              12: 'impala',
              13: 'koribustard',
              14: 'lionfemale',
              15: 'lionmale',
              16: 'otherbird',
              17: 'topi',
              18: 'warthog',
              19: 'wildebeest',
              20: 'zebra'}

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomApply(transforms=[transforms.RandomRotation((90,270))],p=0.4),
        transforms.RandomApply(transforms=[transforms.RandomAffine(degrees=0,translate=(0.1, 0.1))],p=0.1),
        #transforms.Resize(300),
        transforms.RandomResizedCrop(224,scale=(0.5,1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'train_empty': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_empty': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'drive/MyDrive/data/dataset'
image_datasets = {x: SerengetiDataset(os.path.join(data_dir, x + '.csv'), os.path.join(data_dir, x),
                                      data_transforms[x])
                  for x in ['train', 'val']}

data_dir_empty = 'drive/MyDrive/data/empty_dataset'

image_datasets.update({x: datasets.ImageFolder(os.path.join(data_dir_empty, y),
                                               data_transforms[x])
                       for y, x in zip(['train', 'val'], ['train_empty', 'val_empty'])})

dataloaders = {x: DataLoader(image_datasets[x], batch_size=64, shuffle=True, pin_memory=True , num_workers=8)
               for x in ['train', 'val', 'train_empty', 'val_empty']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'train_empty', 'val_empty']}

class_names = list(labels_map.values())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=100, patience=10,
                early_stopping_based_on_loss=True):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #best_empty_acc=0.0
    best_loss = np.Inf
    #epoch_acc_empty=0.0
    list_epoch_loss=[]
    list_average_epoch_acc=[]
    list_running_corrects_specie_first=[]
    list_running_corrects_specie_second=[]
    list_running_corrects_specie_final=[]
    list_running_corrects_majorVoting=[]
    list_epoch_acc_descrizione=[]

    if early_stopping_based_on_loss:
        print('Early stopping based on loss...')
        early_stopping = EarlyStopping(patience=patience, verbose=True)
    else:
        print('Early stopping based on accuracy...')
        early_stopping = EarlyStopping(patience=patience, verbose=True, loss_based=False)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        # TODO Rimuovi train e rinomina variabili
        '''
        if (epoch_acc_empty < 0.55 and epoch > 35)  or epoch < 10:
                torch.save(model.state_dict(), os.path.join('results/models', 'lastbestweightsofempty.pth'))

        if epoch >=10:
            if epoch==10:
                early_stopping.reset()
        '''
        for phase in ['train','val']:
            if phase in ['train']:
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            #running_corrects = 0
            running_corrects_specie_first=0
            running_corrects_specie_second = 0
            running_corrects_specie_final = 0
            running_corrects_majorVoting=0
            #running_corrects_empty=0
            running_corrects_descrizione=0


            # Iterate over data.

            for inputs, specie, descrizione, _ in dataloaders[phase]:
                inputs = inputs.to(device)
                specie = specie.to(device)
                descrizione = descrizione.to(device)
                #emptyimg = emptyimg.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    out_specie_first,out_specie_second,out_specie_final,out_descrizione = model(inputs)

                    #_, preds_emptyimg = torch.max(outputs, 1)
                    _, preds_specie_first = torch.max(out_specie_first, 1)
                    _, preds_specie_second = torch.max(out_specie_second, 1)
                    _, preds_specie_final = torch.max(out_specie_final, 1)
                    _, preds_descrizione = torch.max(out_descrizione, 1)
                    preds_specie_majorVoting=preds_specie_first+preds_specie_second+preds_specie_final

                    #loss1 = criterion(outputs, emptyimg)
                    loss1 = criterion(out_specie_first, specie)
                    loss2 = criterion(out_specie_second, specie)
                    loss3 = criterion(out_specie_final, specie)
                    loss4 = criterion(out_descrizione, descrizione)

                    loss_tot = loss1 + loss2 + loss3+loss4

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_tot.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_tot.item() * inputs.size(0)
                #if phase == 'val':
                running_corrects_specie_first+=torch.sum(preds_specie_first==specie.data)
                running_corrects_specie_second += torch.sum(preds_specie_second == specie.data)
                running_corrects_specie_final += torch.sum(preds_specie_final == specie.data)
                running_corrects_majorVoting+=torch.sum(preds_specie_majorVoting == specie.data)
                running_corrects_descrizione+=torch.sum(preds_descrizione == torch.max(descrizione.data, 1).indices)

                '''
                running_corrects += torch.sum(torch.logical_and(
                    torch.logical_and(preds_emptyimg == emptyimg.data, preds_specie == specie.data),
                    preds_descrizione == torch.max(descrizione.data, 1).indices))
                '''

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc_specie = running_corrects_specie_final.double() / dataset_sizes[phase]
            epoch_acc_specie_MajorVoting=running_corrects_majorVoting.double() / dataset_sizes[phase]
            epoch_acc_descrizione = running_corrects_descrizione.double() / dataset_sizes[phase]
            average_epoch_acc = (epoch_acc_specie_MajorVoting + epoch_acc_descrizione) / 2

            if phase == 'train':
                list_epoch_loss.append(epoch_loss)

            if phase == 'val':
                #epoch_acc_specie=running_corrects_specie_final.double() / dataset_sizes[phase]
                epoch_acc_specie_first = running_corrects_specie_first.double() / dataset_sizes[phase]
                epoch_acc_specie_second = running_corrects_specie_second.double() / dataset_sizes[phase]
                #epoch_acc_specie_MajorVoting = running_corrects_majorVoting.double() / dataset_sizes[phase]
                list_running_corrects_specie_first.append(epoch_acc_specie_first)
                list_running_corrects_specie_second.append(epoch_acc_specie_second)
                list_running_corrects_specie_final.append(epoch_acc_specie)
                list_running_corrects_majorVoting.append(epoch_acc_specie_MajorVoting)


                #epoch_acc_descrizione = running_corrects_descrizione.double() / dataset_sizes[phase]
                #average_epoch_acc=(epoch_acc_specie+epoch_acc_descrizione)/2
                list_epoch_acc_descrizione.append(epoch_acc_descrizione)
                list_average_epoch_acc.append(average_epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f} Acc_Specie_First: {:.4f} Acc_Specie_Second: {:.4f} Acc_Specie_Final: {:.4f} Acc_Specie_MajorVoting: {:.4f} Acc_Descrizione: {:.4f}'.format(phase, epoch_loss, average_epoch_acc,epoch_acc_specie_first,epoch_acc_specie_second,epoch_acc_specie,epoch_acc_specie_MajorVoting,epoch_acc_descrizione))
            else:

                print('{} Loss: {:.4f} Acc: {:.4f} Acc_MajorVoting: {:.4f}'.format(phase, epoch_loss, average_epoch_acc,epoch_acc_specie_MajorVoting))

            # deep copy the model
            if early_stopping_based_on_loss:
                if phase == 'val' and epoch_loss < best_loss:
                    best_acc = average_epoch_acc
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(),os.path.join('drive/MyDrive/results/models','lastbestweightsLightBranchesResNet34Backbone.pth'))
                    print('Better val loss')
            else:
                if phase == 'val' and average_epoch_acc > best_acc:
                    best_acc = average_epoch_acc
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), os.path.join('drive/MyDrive/results/models', 'lastbestweightsLightBranchesResNet34Backbone.pth'))

                    print('Better val accuracy')

        print()



        if early_stopping_based_on_loss:
            early_stopping(epoch_loss, model)
        else:
            early_stopping(average_epoch_acc, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    time_elapsed = time.time() - since
    print(list_epoch_loss)
    print(list_average_epoch_acc)
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f} with Acc of {:4f}'.format(best_loss, best_acc))

    # load best model weights
    '''
    f = open('lossResnet34withBracnhes.txt', 'w')
    for ele in list_epoch_loss:
        f.write(ele + '\n')

    f.close()
    f = open('averageEpochAccResnet34withBracnhes.txt', 'w')
    for ele in list_average_epoch_acc:
        f.write(ele + '\n')

    f.close()
    f = open('MajorVotingAccResnet34withBracnhes.txt', 'w')
    for ele in list_running_corrects_majorVoting:
        f.write(ele + '\n')

    f.close()
    f = open('FirstOutputAccResnet34withBracnhes.txt', 'w')
    for ele in list_running_corrects_specie_first:
        f.write(ele + '\n')

    f.close()
    f = open('SecondOutputAccResnet34withBracnhes.txt', 'w')
    for ele in list_running_corrects_specie_second:
        f.write(ele + '\n')

    f.close()
    f = open('FinalOutputAccResnet34withBracnhes.txt', 'w')
    for ele in list_running_corrects_specie_final:
        f.write(ele + '\n')

    f.close()
    f = open('DescriptionAccResnet34withBracnhes.txt', 'w')
    for ele in list_epoch_acc_descrizione:
        f.write(ele + '\n')

    f.close()
    '''
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
modelmod34.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(modelmod34.parameters(), lr=0.1, momentum=0.9)


# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

In [ ]:
modelmod34=train_model(modelmod34, criterion, optimizer_ft, exp_lr_scheduler,
                           patience=7,num_epochs=60, early_stopping_based_on_loss=False)
torch.save(modelmod34, os.path.join('drive/MyDrive/results/models', '34BackboneWithBranches.pth'))


Early stopping based on accuracy...
Epoch 0/99
----------
train Loss: 31.4524 Acc: 0.2583 Acc_MajorVoting: 0.0111


RuntimeError: ignored

# Backbone50

### Backbone50withoutBranches

In [ ]:
class Backbone50(nn.Module):
  def __init__(self,block,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=256,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      #self.LightClassificationBranch1=LightClassificationBranch(inplanes=256,num_classes=num_classes)
      
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      #self.LightClassificationBranch2=LightClassificationBranch(inplanes=512,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer31=self._make_layer(block,256,layers[2])
      self.layer41=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      self.fc1=nn.Linear(512*block.expansion,num_classes)
      self.inplanes=1024
      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)          # 56x56
      
      x = self.layer2(x)   
             # 28x28
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)
      #x1=torch.stack((x1,feature_vector1,feature_vector2),axis=1)
      #x1=torch.sum(x1,axis=1) 
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      x2=self.fc2(x2)


      return x1,x2

In [ ]:
backbone50=backbone50()
backbone50

In [ ]:
a=backbone50.conv1.weight
b=resntet50pre.conv1.weight
torch.equal(a,b)

True

In [ ]:
x1,x2=backbone50(inp)

In [ ]:
initializeWeights50(backbone50)

#### Training

In [ ]:
labels_map = {0: 'baboon',
              1: 'buffalo',
              2: 'cheetah',
              3: 'eland',
              4: 'elephant',
              5: 'empty',
              6: 'gazellegrants',
              7: 'gazellethomsons',
              8: 'giraffe',
              9: 'guineafowl',
              10: 'hartebeest',
              11: 'hyenaspotted',
              12: 'impala',
              13: 'koribustard',
              14: 'lionfemale',
              15: 'lionmale',
              16: 'otherbird',
              17: 'topi',
              18: 'warthog',
              19: 'wildebeest',
              20: 'zebra'}

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(p=0.3),
        T.RandomApply(transforms=[T.RandomRotation((30))], p=0.3),
        T.RandomApply(transforms=[T.GaussianBlur(kernel_size=(7), sigma=(1))], p=0.1),
        # transforms.Resize(300),
        transforms.RandomResizedCrop(224, scale=(0.5, 1)),
        transforms.RandomAdjustSharpness(sharpness_factor=2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'train_empty': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(p=0.3),
        T.RandomApply(transforms=[T.RandomRotation((30))], p=0.3),
        T.RandomApply(transforms=[T.GaussianBlur(kernel_size=(7), sigma=(1))], p=0.1),
        # transforms.Resize(300),
        transforms.RandomResizedCrop(224, scale=(0.5, 1)),
        transforms.RandomAdjustSharpness(sharpness_factor=2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_empty': transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'drive/MyDrive/data/dataset'
image_datasets = {x: SerengetiDataset(os.path.join(data_dir, x + '.csv'), os.path.join(data_dir, x),
                                      data_transforms[x])
                  for x in ['train', 'val']}

data_dir_empty = 'drive/MyDrive/data/empty_dataset'

image_datasets.update({x: datasets.ImageFolder(os.path.join(data_dir_empty, y),
                                               data_transforms[x])
                       for y, x in zip(['train', 'val'], ['train_empty', 'val_empty'])})

dataloaders = {x: DataLoader(image_datasets[x], batch_size=128, shuffle=True, pin_memory=True , num_workers=8)
               for x in ['train', 'val', 'train_empty', 'val_empty']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'train_empty', 'val_empty']}

class_names = list(labels_map.values())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=100, patience=10,
                early_stopping_based_on_loss=False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #best_empty_acc=0.0
    best_loss = np.Inf
    #epoch_acc_empty=0.0
    list_epoch_loss=[]
    list_average_epoch_acc=[]
    #list_running_corrects_specie_first=[]
    #list_running_corrects_specie_second=[]
    list_epoch_acc_specie=[]

    list_epoch_acc_descrizione=[]

    if early_stopping_based_on_loss:
        print('Early stopping based on loss...')
        early_stopping = EarlyStopping(patience=patience, verbose=True)
    else:
        print('Early stopping based on accuracy...')
        early_stopping = EarlyStopping(patience=patience, verbose=True, loss_based=False)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        # TODO Rimuovi train e rinomina variabili
        '''
        if (epoch_acc_empty < 0.55 and epoch > 35)  or epoch < 10:
                torch.save(model.state_dict(), os.path.join('results/models', 'lastbestweightsofempty.pth'))

        if epoch >=10:
            if epoch==10:
                early_stopping.reset()
        '''
        for phase in ['train','val']:
            if phase in ['train']:
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects_specie = 0
            running_corrects_descrizione=0


            # Iterate over data.

            for inputs, specie, descrizione, _ in dataloaders[phase]:
                inputs = inputs.to(device)
                specie = specie.to(device)
                descrizione = descrizione.to(device)
                #emptyimg = emptyimg.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    out_specie,out_descrizione = model(inputs)

                    #_, preds_emptyimg = torch.max(outputs, 1)

                    _, preds_specie = torch.max(out_specie, 1)
                    _, preds_descrizione = torch.max(out_descrizione, 1)


                    #loss1 = criterion(outputs, emptyimg)
                    loss1 = criterion(out_specie, specie)

                    loss2 = criterion(out_descrizione, descrizione)

                    loss_tot = loss1 + loss2

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_tot.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_tot.item() * inputs.size(0)
                #if phase == 'val':
                running_corrects_specie+=torch.sum(preds_specie==specie.data)
                running_corrects_descrizione+=torch.sum(preds_descrizione == torch.max(descrizione.data, 1).indices)

                '''
                running_corrects += torch.sum(torch.logical_and(
                    torch.logical_and(preds_emptyimg == emptyimg.data, preds_specie == specie.data),
                    preds_descrizione == torch.max(descrizione.data, 1).indices))
                '''

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc_specie = running_corrects_specie.double() / dataset_sizes[phase]

            epoch_acc_descrizione = running_corrects_descrizione.double() / dataset_sizes[phase]
            average_epoch_acc = (epoch_acc_specie + epoch_acc_descrizione) / 2

            if phase == 'train':
                list_epoch_loss.append(epoch_loss)

            if phase == 'val':
                #epoch_acc_descrizione = running_corrects_descrizione.double() / dataset_sizes[phase]
                #average_epoch_acc=(epoch_acc_specie+epoch_acc_descrizione)/2
                list_epoch_acc_specie.append(epoch_acc_specie)
                list_epoch_acc_descrizione.append(epoch_acc_descrizione)
                list_average_epoch_acc.append(average_epoch_acc)



            print('{} Loss: {:.4f} Acc: {:.4f} Acc_Specie: {:.4f} Acc_Descrizione: {:.4f}'.format(phase, epoch_loss, average_epoch_acc,epoch_acc_specie,epoch_acc_descrizione))

            # deep copy the model
            if early_stopping_based_on_loss:
                if phase == 'val' and epoch_loss < best_loss:
                    best_acc = average_epoch_acc
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(),os.path.join('drive/MyDrive/results/models','lastbestweights50Backbone.pth'))
                    print('Better val loss')
            else:
                if phase == 'val' and average_epoch_acc > best_acc:
                    best_acc = average_epoch_acc
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), os.path.join('drive/MyDrive/results/models', 'lastbestweights50Backbone.pth'))

                    print('Better val accuracy')

        print()



        if early_stopping_based_on_loss:
            early_stopping(epoch_loss, model)
        else:
            early_stopping(average_epoch_acc, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    time_elapsed = time.time() - since
    print(list_epoch_loss)
    print(list_average_epoch_acc)
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f} with Acc of {:4f}'.format(best_loss, best_acc))

    # load best model weights
    '''
    f = open('lossBackbone34.txt', 'w')
    for ele in list_epoch_loss:
        f.write(ele + '\n')

    f.close()
    f = open('averageEpochAccBackbone34.txt', 'w')
    for ele in list_average_epoch_acc:
        f.write(ele + '\n')

    f.close()

    f = open('AccSpecieBackbone34.txt', 'w')
    for ele in list_epoch_acc_specie:
        f.write(ele + '\n')

    f.close()


    f = open('DescriptionAccBackbone34.txt', 'w')
    for ele in list_epoch_acc_descrizione:
        f.write(ele + '\n')

    f.close()
    '''
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
backbone50.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(backbone50.parameters(), lr=0.01, momentum=0.9)


# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)


In [ ]:
backbone50=train_model(backbone50, criterion, optimizer_ft, exp_lr_scheduler,
                           patience=7,num_epochs=10, early_stopping_based_on_loss=False)
torch.save(modelmod34, os.path.join('drive/MyDrive/results/models', '50BackboneWithouthBranches.pth'))


Early stopping based on accuracy...
Epoch 0/9
----------
train Loss: 3.5411 Acc: 0.3789 Acc_Specie: 0.1758 Acc_Descrizione: 0.5820


KeyboardInterrupt: ignored

### Backbone50withAddBranches

In [ ]:
class Backbone50withAddBranches(nn.Module):
  def __init__(self,block,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=256,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      self.LightClassificationBranch1=LightClassificationBranch(inplanes=256,num_classes=num_classes)
      
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      self.LightClassificationBranch2=LightClassificationBranch(inplanes=512,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer31=self._make_layer(block,256,layers[2])
      self.layer41=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      self.fc1=nn.Linear(512*block.expansion,num_classes)
      self.inplanes=1024
      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)          # 56x56
      featurevector1,out1=self.LightClassificationBranch1(x)
      x = self.layer2(x)   
      featurevector2,out2=self.LightClassificationBranch2(x)       # 28x28
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)
      x1=torch.stack((x1,featurevector1,featurevector2),axis=1)
      x1=torch.sum(x1,axis=1)
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      x2=self.fc2(x2)


      return out1,out2,x1,x2

In [ ]:
backbone50addbranches=backbone50withAddBranches()
backbone50addbranches

In [ ]:
out1,out2,x1,x2=backbone50addbranches(inp)

In [ ]:
x2.shape

torch.Size([2, 3])

### Backbone50withBranches

In [ ]:
class ResNetMod(nn.Module):
  def __init__(self,block,layers,num_classes=21):
      super().__init__()

      self.inplanes=64
      self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
      self.bn1=nn.BatchNorm2d(self.inplanes)
      self.relu=nn.ReLU(inplace=True)
      self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
      #self.classificationBranch=ClassificationBranch(inplanes=256,planes=128)
      
      self.layer1=self._make_layer(block,64,layers[0])
      self.LightClassificationBranch1=LightClassificationBranch(inplanes=256,num_classes=num_classes)
      
      self.layer2=self._make_layer(block,128,layers[1],stride=2)
      self.LightClassificationBranch2=LightClassificationBranch(inplanes=512,num_classes=num_classes)
      self.layer3=self._make_layer(block,256,1,stride=2)

      self.layer31=self._make_layer(block,256,layers[2])
      self.layer41=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool1=nn.AdaptiveAvgPool2d((1,1))
      self.fc1=nn.Linear(512*block.expansion,num_classes)
      self.inplanes=1024
      self.layer32=self._make_layer(block,256,layers[2])
      self.layer42=self._make_layer(block,512,layers[3],stride=2)
      self.avgpool2=nn.AdaptiveAvgPool2d((1,1))
      self.fc2=nn.Linear(512*block.expansion,3)


  def _make_layer(self, block, planes, blocks, stride=1):
      downsample = None  
  
      if stride != 1 or self.inplanes != planes*block.expansion:
          downsample = nn.Sequential(
              nn.Conv2d(self.inplanes, planes*block.expansion, 1, stride, bias=False),
              nn.BatchNorm2d(planes*block.expansion),
          )

      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      
      self.inplanes = planes*block.expansion
      
      for _ in range(1, blocks):
          layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)

  def forward(self,x):
      x=self.conv1(x)
      x=self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)         # 112x112
      
      x = self.layer1(x)          # 56x56
      featurevector1,out1=self.LightClassificationBranch1(x)
      x = self.layer2(x)   
      featurevector2,out2=self.LightClassificationBranch2(x)       # 28x28
      x = self.layer3(x)          # 14x14
      
      x1 = self.layer31(x)
      x1= self.layer41(x1)          # 7x7
      x1 = self.avgpool1(x1)         # 1x1
      x1 = torch.flatten(x1, 1)
      x1=torch.stack((x1,feature_vector1,feature_vector2),axis=1)
      x1=torch.sum(x1,axis=1) 
      x1 = self.fc1(x1)
      
      x2= self.layer32(x)
      x2=self.layer42(x2)
      x2=self.avgpool2(x2)
      x2=torch.flatten(x2,1)
      x2=self.fc2(x2)


      return out1,out2,x1,x2

In [ ]:

modelmod50=resnetmod50()
modelmod50

In [ ]:
#output=modelmod50(inp)
v1,v2,final,desc=modelmod50(inp)
desc.shape

torch.Size([2, 3])

# Metodi per creazioni reti

In [ ]:
def resnet34():
    layers=[3, 4, 6, 3]
    model = ResNet(BasicBlock, layers)
    return model

def resnet50():
  layers=[3,4,6,3]
  model= ResNet(Bottleneck,layers)
  return model

def backbone50():
  layers=[3,4,5,3]
  model= Backbone50(Bottleneck,layers)
  return model


def resnetmod50():
  layers=[3,4,5,3]
  model= ResNetMod(Bottleneck,layers)
  return model

def backbone50withAddBranches():
  layers=[3,4,5,3]
  model= Backbone50withAddBranches(Bottleneck,layers)
  return model  

def backbone34():
  layers=[3,4,5,3,4,6,3] 
  model=Backbone34(BasicBlock,Bottleneck,layers)
  return model   

def resnetmod34():
  layers=[3,4,5,3,4,6,3] 
  model=Backbone34(BasicBlock,Bottleneck,layers)
  return model 

def resnetmodmod34():
  layers=[3,4,5,3,4,6,3] 
  model=ResNetModmod34(BasicBlock,Bottleneck,layers)
  return model  

def backbone34withAddBranches():
  layers=[3,4,5,3,4,6,3]
  model=ResNetModmodAdd34(BasicBlock,Bottleneck,layers)
  return model








# Weights Initialization

In [ ]:
resntet50pre=models.resnet50(pretrained=True)
resntet50pre


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
resntet50pre.bn1.state_dict()

OrderedDict([('weight',
              tensor([2.3888e-01, 2.9136e-01, 3.1615e-01, 2.7122e-01, 2.1731e-01, 3.0903e-01,
                      2.2937e-01, 2.3086e-01, 2.1129e-01, 2.8054e-01, 1.9923e-01, 3.1894e-01,
                      1.7955e-01, 1.1246e-08, 1.9704e-01, 2.0996e-01, 2.4317e-01, 2.1697e-01,
                      1.9415e-01, 3.1569e-01, 1.9648e-01, 2.3214e-01, 2.1962e-01, 2.1633e-01,
                      2.4357e-01, 2.9683e-01, 2.3852e-01, 2.1162e-01, 1.4492e-01, 2.9388e-01,
                      2.2911e-01, 9.2716e-02, 4.3334e-01, 2.0782e-01, 2.7990e-01, 3.5804e-01,
                      2.9315e-01, 2.5306e-01, 2.4210e-01, 2.1755e-01, 3.8645e-01, 2.1003e-01,
                      3.6805e-01, 3.3724e-01, 5.0826e-01, 1.9341e-01, 2.3914e-01, 2.6652e-01,
                      3.9020e-01, 1.9840e-01, 2.1694e-01, 2.6666e-01, 4.9806e-01, 2.3553e-01,
                      2.1349e-01, 2.5951e-01, 2.3547e-01, 1.7579e-01, 4.5354e-01, 1.7102e-01,
                      2.4903e-01, 2.

In [ ]:

resnetpre=models.resnet34(pretrained=True)
resnetpre


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
torch.save(resnetpre.conv1.state_dict(),'resnet34conv1.pth')
torch.save(resnetpre.layer1.state_dict(),'resnet34layer1.pth')
torch.save(resnetpre.layer2.state_dict(),'resnet34layer2.pth')
torch.save(resnetpre.layer3.state_dict(),'resnet34layer3.pth')
torch.save(resnetpre.layer4.state_dict(),'resnet34layer4.pth')
torch.save(resnetpre.fc.state_dict(),'resnet34layerfc.pth')
torch.save(resntet50pre.conv1.state_dict(),'resnet50conv1.pth')
torch.save(resntet50pre.layer1.state_dict(),'resnet50layer1.pth')
torch.save(resntet50pre.layer2.state_dict(),'resnet50layer2.pth')
torch.save(resntet50pre.layer3.state_dict(),'resnet50layer3.pth')
torch.save(resntet50pre.layer4.state_dict(),'resnet50layer4.pth')


In [ ]:
def initializeWeights34(modelmod34):
  modelmod34.conv1.load_state_dict(torch.load('resnet34conv1.pth'))
  modelmod34.layer1.load_state_dict(torch.load('resnet34layer1.pth'))
  modelmod34.layer2.load_state_dict(torch.load('resnet34layer2.pth'))
  modelmod34.layer42.load_state_dict(torch.load('resnet34layer4.pth'))
  modelmod34.layer31.load_state_dict(torch.load('resnet50layer2.pth'))
  modelmod34.layer41.load_state_dict(torch.load('resnet50layer3.pth'))
  modelmod34.layer51.load_state_dict(torch.load('resnet50layer4.pth'))


In [ ]:
def initializeWeights50(modelmod50):
  modelmod50.conv1.load_state_dict(torch.load('resnet50conv1.pth'))
  modelmod50.layer1.load_state_dict(torch.load('resnet50layer1.pth'))
  modelmod50.layer2.load_state_dict(torch.load('resnet50layer2.pth'))
  modelmod50.layer41.load_state_dict(torch.load('resnet50layer4.pth'))
  modelmod50.layer42.load_state_dict(torch.load('resnet50layer4.pth'))
  

# GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 21 14:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------